### References:

- https://discourse.julialang.org/t/slow-arbitrary-base-exponentiation-a-b/25386/21
- https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations-1
- http://llvm.org/docs/LangRef.html#fast-math-flags

In [10]:
using Revise, BernsteinExpansions, DynamicPolynomials, BenchmarkTools, StaticArrays, Test

In [11]:
using LoopVectorization # try @avx macro

In [12]:
using BernsteinExpansions: fastpow # DiffEqBase implementation

In [13]:
# https://github.com/etheory/fastapprox/blob/master/fastapprox/src/fastlog.h
function fastlog2(x::Float32)::Float32
    y = Float32(reinterpret(Int32, x))
    y *= 1.1920928955078125f-7
    y - 126.94269504f0
end
function fastlog2(x::Float64)::Float32
   fastlog2(Float32(x))
end

# https://github.com/etheory/fastapprox/blob/master/fastapprox/src/fastexp.h
function _fastpow2(x::Float32)::Float32
    clipp = x < -126.0f0 ? -126.0f0 : x
    clipp = min(126f0, max(-126f0, x))
    reinterpret(Float32, UInt32((1 << 23) * (clipp + 126.94269504f0)))
end
function _fastpow2(x::Float64)::Float32
   _fastpow2(Float32(x))
end

# https://github.com/etheory/fastapprox/blob/master/fastapprox/src/fastpow.h
function fastpow2(x::Real, y::Real)::Real
    _fastpow2(y * fastlog2(x))
end

fastpow2 (generic function with 1 method)

In [14]:
@btime fastpow2(2.0, 4)

  0.017 ns (0 allocations: 0 bytes)


18.750488f0

In [15]:
@btime fastpow(2.0, 4)

  0.016 ns (0 allocations: 0 bytes)


16.0f0

In [16]:
Float64(fastpow2(2.0, 4))

18.75048828125

In [17]:
Float64(fastpow(2.0, 4))

16.0

In [18]:
function uni_1(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    for i in 0:l
        coeffs[i+1] = low^(k-i) * high^i
    end
    return coeffs
end

# using Julia's ^, no fastmath
function uni_2(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    for i in 0:l
        @inbounds coeffs[i+1] = low^(k-i) * high^i
    end
    return coeffs
end

function uni_3(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = zeros(N, l+1)
    @simd for i in 0:l
        @inbounds coeffs[i+1] = low^(k-i) * high^i
    end
    return coeffs
end

# using Julia's ^ with fastmath
function uni_4(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    @fastmath for i in 0:l
        @inbounds coeffs[i+1] = low^(k-i) * high^i
    end
    return coeffs
end

function uni_6(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = MVector{l+1, Float64}(undef)
    @fastmath for i in 0:l
        @inbounds coeffs[i+1] = low^(k-i) * high^i
    end
    return coeffs
end

# using fastpow and fastmath
function uni_8(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    @fastmath for i in 0:l
        @inbounds coeffs[i+1] = fastpow(low, k-i) * fastpow(high, i)
    end
    return coeffs
end

# using fastpow and not fastmath
function uni_9(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    for i in 0:l
        @inbounds coeffs[i+1] = fastpow(low, k-i) * fastpow(high, i)
    end
    return coeffs
end

function uni_10(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    for i in 0:l
        @inbounds coeffs[i+1] = fastpow2(low, k-i) * fastpow2(high, i)
    end
    return coeffs
end

uni_10 (generic function with 1 method)

In [19]:
@btime uni_1(3, 3, 1.0, 2.0)
@btime uni_2(3, 3, 1.0, 2.0) # using Julia's ^, no fastmath
@btime uni_3(3, 3, 1.0, 2.0)
@btime uni_4(3, 3, 1.0, 2.0) # using Julia's ^ and fastmath << FASTEST
@btime uni_6(3, 3, 1.0, 2.0)

  98.267 ns (1 allocation: 112 bytes)
  99.671 ns (1 allocation: 112 bytes)
  104.276 ns (1 allocation: 112 bytes)
  37.174 ns (1 allocation: 112 bytes)
  37.703 ns (1 allocation: 112 bytes)
  1.026 μs (18 allocations: 816 bytes)


4-element MArray{Tuple{4},Float64,1,4} with indices SOneTo(4):
 1.0
 2.0
 4.0
 8.0

In [20]:
@btime uni_8(3, 3, 1.0, 2.0) # using fastpow and fastmath 
@btime uni_9(3, 3, 1.0, 2.0) # using fastpow and NOT fastmath << twice as slow than uni_4, *but* faster than uni_2
@btime uni_10(3, 3, 1.0, 2.0)

  37.330 ns (1 allocation: 112 bytes)
  79.859 ns (1 allocation: 112 bytes)
  81.294 ns (1 allocation: 112 bytes)
  58.221 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
 1.0826728343963623
 2.1146087646484375
 4.229248046875    
 8.661382675170898 

In [21]:
using LoopVectorization

In [61]:
function uni_12(k::Integer, l::Integer, low::N, high::N) where {N}
    coeffs = Vector{N}(undef, l+1)
    @fastmath low_k = low^k
    α = high / low
    αⁱ = 1 # assuming low is not zero
    @inbounds for i in 0:l
        coeffs[i+1] = low_k * αⁱ
        αⁱ = αⁱ * α
    end
    return coeffs
end

uni_12 (generic function with 1 method)

In [62]:
@btime uni_12(3, 3, 1.0, 2.0)

  26.032 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
 1.0
 2.0
 4.0
 8.0

In [73]:
@btime uni_12(10, 10, 1.0, 2.0)

  31.402 ns (1 allocation: 176 bytes)


11-element Array{Float64,1}:
    1.0
    2.0
    4.0
    8.0
   16.0
   32.0
   64.0
  128.0
  256.0
  512.0
 1024.0

In [74]:
@btime uni_4(10, 10, 1.0, 2.0)

  72.662 ns (1 allocation: 176 bytes)


11-element Array{Float64,1}:
    1.0
    2.0
    4.0
    8.0
   16.0
   32.0
   64.0
  128.0
  256.0
  512.0
 1024.0

In [77]:
@btime uni_4(3, 2, 0.2, 1.5)

  33.230 ns (1 allocation: 112 bytes)


3-element Array{Float64,1}:
 0.008000000000000002
 0.06000000000000001 
 0.45                

In [79]:
using DynamicPolynomials
@polyvar x

(x,)

In [80]:
m = x^3

x³

In [83]:
dom = 0.2..2.5

@btime univariate($m, 3, $dom)

  42.444 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
  0.007999999999999998
  0.09999999999999998 
  1.25                
 15.625               

In [86]:
@btime uni_12(3, 3, inf($dom), sup($dom))

  26.088 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
  0.007999999999999998
  0.09999999999999999 
  1.2500000000000002  
 15.625000000000004   

In [92]:
@polyvar y

p = 1 - x^4*y + x^5

x⁵ - x⁴y + 1

In [93]:
for t in terms(p)
    println(nvariables(t))
end

2
2
2


In [100]:
multivariate(terms(p)[1], [3, 3], IntervalBox(1..2, 2))

2-element Array{Array{Float64,1},1}:
 [1.0, 2.0, 4.0, 8.0]
 [1.0, 1.0, 1.0, 1.0]

In [101]:
BernsteinExpansions.assemble(ans)

4×4 Array{Float64,2}:
 1.0  1.0  1.0  1.0
 2.0  2.0  2.0  2.0
 4.0  4.0  4.0  4.0
 8.0  8.0  8.0  8.0

In [134]:
using BernsteinExpansions: assemble

function f(p)
    x = multivariate(terms(p)[1], [5, 5], IntervalBox(1..2, 2))
    assemble(x)
end

f (generic function with 1 method)

In [135]:
@btime f($p)

  444.556 ns (12 allocations: 1.25 KiB)


6×6 Array{Float64,2}:
  1.0   1.0   1.0   1.0   1.0   1.0
  2.0   2.0   2.0   2.0   2.0   2.0
  4.0   4.0   4.0   4.0   4.0   4.0
  8.0   8.0   8.0   8.0   8.0   8.0
 16.0  16.0  16.0  16.0  16.0  16.0
 32.0  32.0  32.0  32.0  32.0  32.0

In [136]:
terms(p)[1]

x⁵

In [137]:
exponents(ans)

2-element Array{Int64,1}:
 5
 0

In [141]:
a = uni_12(5, 5, 1.0, 2.0)

6-element Array{Float64,1}:
  1.0
  2.0
  4.0
  8.0
 16.0
 32.0

In [142]:
b = uni_12(0, 5, 1.0, 2.0)

6-element Array{Float64,1}:
  1.0
  2.0
  4.0
  8.0
 16.0
 32.0

In [132]:
uni_2(0, 3, 1.0, 2.0)

4-element Array{Float64,1}:
 1.0
 2.0
 4.0
 8.0

In [140]:
univariate(x^5, 5, 1..2)

6-element Array{Float64,1}:
  1.0
  2.0
  4.0
  8.0
 16.0
 32.0

In [133]:
exponents(x^0)

1-element Array{Int64,1}:
 0

In [124]:
multivariate(terms(p)[1], [3, 3], IntervalBox(1..2, 2))

2-element Array{Array{Float64,1},1}:
 [1.0, 2.0, 4.0, 8.0]
 [1.0, 1.0, 1.0, 1.0]

## Test

In [2]:
using Revise, BernsteinExpansions, DynamicPolynomials, BenchmarkTools, StaticArrays, Test

┌ Info: Precompiling BernsteinExpansions [afc6e731-2783-597c-a3b4-470ee96642a0]
└ @ Base loading.jl:1273


In [3]:
@polyvar x
m = x^3

x³

In [6]:
@btime univariate($m, 3, 1..2)

  43.479 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
 1.0
 2.0
 4.0
 8.0

In [8]:
m = x^5
@btime univariate($m, 2, 0..1)

  39.891 ns (1 allocation: 112 bytes)


3-element Array{Float64,1}:
 0.0
 0.0
 0.0

In [13]:
@btime univariate(x^2, 2, 0..4)

  105.720 ns (4 allocations: 336 bytes)


3-element Array{Float64,1}:
  0.0
  0.0
 16.0

In [28]:
coeffs = Vector{Float64}(undef, 4)
using BernsteinExpansions: _univariate!

In [29]:
@btime _univariate!($coeffs, 4, 3, 1.0, 4.0, Val(:fastmath))
@btime _univariate!($coeffs, 4, 3, 1.0, 4.0, Val(:fastpow))
@btime _univariate!($coeffs, 4, 3, 1.0, 4.0, Val(:base))

  21.285 ns (0 allocations: 0 bytes)
  63.144 ns (0 allocations: 0 bytes)
  87.559 ns (0 allocations: 0 bytes)


4-element Array{Float64,1}:
  1.0
  4.0
 16.0
 64.0

In [26]:
m = x^4
@btime univariate($m, 3, 1..4)

  43.859 ns (1 allocation: 112 bytes)


4-element Array{Float64,1}:
  1.0
  4.0
 16.0
 64.0

In [27]:
univariate(x^4, 3, 1..4)

4-element Array{Float64,1}:
  1.0
  4.0
 16.0
 64.0

In [31]:
@which 2.0^4

^(x::Float64, y::Integer) in Base.Math at math.jl:860